In [54]:
import numpy as np
import pandas as pd
import json
import os
import requests
from datetime import datetime , timedelta
import matplotlib.pyplot as plt

In [39]:
path =  "C:/Users/std92/OneDrive/文件/AMM/other's notes and codes/token_style/Data/LP data/WETH_USDC_arbitrum_500_lp.csv"
df = pd.read_csv(path)[[ 'timestamp', 'blockNumber','event','amount0','amount1','amount','tickLower','tickUpper']]

In [40]:
df[['amount0','amount1','amount']]=df[['amount0','amount1','amount']].astype(float)
df['amount0'] = df['amount0'].apply(lambda r: r*10**(-16))
df['amount1'] = df['amount1'].apply(lambda r: r*10**(-8))
df

,timestamp,blockNumber,event,amount0,amount1,amount,tickLower,tickUpper
0,1626122625,100909,Mint,0.809232,0.227511,3.632972e+12,-203190,-198080
1,1630408764,227288,Mint,0.100000,0.000000,9.198078e+11,-194710,-193380
2,1630452896,228713,Mint,5.754390,1.000000,4.813079e+15,-200200,-200180
3,1630455648,228962,Mint,0.130091,0.033719,2.299615e+11,-200310,-186450
4,1630458698,229112,Mint,28.516789,13.468888,2.734891e+14,-196370,-193360
...,...,...,...,...,...,...,...,...
248662,1697646401,141750154,Mint,23.681484,1.293320,1.913957e+14,-203060,-201710
248663,1697646417,141750210,Mint,1.599000,0.000000,2.111565e+13,-202710,-202100
248664,1697646582,141750800,Mint,1479.723200,767.361689,5.045673e+18,-202730,-202720
248665,1697646652,141751048,Mint,2035.335245,680.028176,5.045565e+18,-202730,-202720


In [60]:
# upper lower price
df[['tickLower','tickUpper']].apply(lambda c : 1.0001**c * 10**12)

,tickLower,tickUpper
0,1499.724942,2499.913599
1,3501.667635,3999.753253
2,2022.363284,2026.411856
3,2000.240294,7998.052102
4,2966.099000,4007.760364
...,...,...
248662,1519.347656,1738.937544
248663,1573.463624,1672.427677
248664,1570.319999,1571.891025
248665,1570.319999,1571.891025


In [63]:
df[df.amount == 0]

,timestamp,blockNumber,event,amount0,amount1,amount,tickLower,tickUpper
24,1630489475,237363,Burn,0.0,0.0,0.0,-207240,-186450
29,1630497812,239868,Burn,0.0,0.0,0.0,-204210,-185900
32,1630498034,239903,Burn,0.0,0.0,0.0,-204200,-186010
34,1630498152,239926,Burn,0.0,0.0,0.0,-204180,-186010
38,1630498171,239951,Burn,0.0,0.0,0.0,-204180,-186010
...,...,...,...,...,...,...,...,...
210068,1686656478,100726844,Burn,0.0,0.0,0.0,-202760,-200110
213467,1687696059,104764082,Burn,0.0,0.0,0.0,-201830,-200020
217488,1689089898,110156888,Burn,0.0,0.0,0.0,-205940,-195930
217489,1689089898,110156888,Burn,0.0,0.0,0.0,-212940,-200940


### Some codes for processing the raw data

In [ ]:
def resample_data(df, k):
    new_df = pd.DataFrame(columns=['blockNumber', 'timestamp', 'tick', 'price', 'WETH_buy', 'USDC_buy', 'WETH_sell', 'USDC_sell'])
    new_df.set_index('blockNumber', inplace=True)

    for block_start in range(df.iloc[0].name, df.iloc[-1].name, k):
        block_end = block_start + k
        df_interval = df[(df.index >= block_start) & (df.index < block_end)]

        if not df_interval.empty:
            new_timestamp = get_timestamp_from_blocknumber(block_end)
            if new_timestamp is not None:
                new_df.loc[block_end] = [
                    new_timestamp,
                    df_interval.iloc[-1]['tick'],
                    df_interval.iloc[-1]['price'],
                    df_interval[df_interval['WETH'] >= 0]['WETH'].sum(),
                    df_interval[df_interval['WETH'] >= 0]['USDC'].sum(),
                    df_interval[df_interval['WETH'] <= 0]['WETH'].sum(),
                    df_interval[df_interval['WETH'] <= 0]['USDC'].sum()
                ]

    return new_df
def get_timestamp_from_blocknumber(block_number):
    """
    Function to get the timestamp for a given block number on Arbitrum.

    Args:
        block_number (int): The block number on Arbitrum.

    Returns:
        timestamp (int): The timestamp corresponding to the given block number.
    """

# Arbitrum Node API endpoint
    arbitrum_node_url = "https://arb-mainnet.g.alchemy.com/v2/nHzD3Ofjd2yRam6T9HYBjgtIqp2l8i2K"  # Replace with your Alchemy API key

# JSON-RPC payload
    payload = {
        "jsonrpc": "2.0",
        "method": "eth_getBlockByNumber",
        "params": [hex(block_number), False],
        "id": 1
    }

    try:
        # Send the request to the Arbitrum Node API
        response = requests.post(arbitrum_node_url, json=payload)
        response.raise_for_status()

# Extract the timestamp from the response
        timestamp = int(response.json()["result"]["timestamp"], 16)

        return timestamp

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None